In [4]:
# Import ray serve and FastAPI libraries
import ray
from ray import serve
from fastapi import FastAPI
import requests 

# library for pre-trained models
from transformers import pipeline

In [2]:
# Define a simple FastAPI app
app = FastAPI()

# Define a Ray Serve deployment
# This decorator registers the class as a Ray Serve deployment
@serve.deployment(num_replicas=2) # num_replicas specifies the number of replicas for load balancing
@serve.ingress(app) # This decorator allows the FastAPI app to be served by Ray Serve
class MySentimentModel:
    def __init__(self):
        # Load a pre-trained sentiment analysis model
        self.model = pipeline("sentiment-analysis",
                              model="distilbert-base-uncased-finetuned-sst-2-english")

    # Define any necessary application logic or transformation logic
    def application_logic(self, text):
        """        Apply any necessary application logic to the input text.
        """
        # simple application logic: truncate text if it exceeds a certain length
        if len(text) > 50:
            return text[:50].lower()  # Truncate and convert to lowercase
        else:
            return text.lower()
        
    @app.get("/predict") # Define an endpoint for predictions
    def predict(self, text: str):
        """        Predict sentiment for the given text.
        """
        # Define any necessary application logic or transformation logic
        text = self.application_logic(text) # Apply any necessary application logic to the input text

        # Use the model to make a prediction
        result = self.model(text)
        return {"text": text, "sentiment": result}

In [3]:
serve.run(MySentimentModel.bind())

2025-08-02 16:33:04,056	INFO worker.py:1918 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(ProxyActor pid=60476) INFO 2025-08-02 16:33:06,865 proxy 127.0.0.1 -- Proxy starting on node 8a746934481c99344a7fa35fccaa9d0823d0a71116ac4b043ca84bb2 (HTTP port: 8000).
(ProxyActor pid=60476) INFO 2025-08-02 16:33:06,913 proxy 127.0.0.1 -- Got updated endpoints: {}.
INFO 2025-08-02 16:33:07,204 serve 57504 -- Started Serve in namespace "serve".
(ServeController pid=60472) INFO 2025-08-02 16:33:10,018 controller 60472 -- Deploying new version of Deployment(name='MySentimentModel', app='default') (initial target replicas: 2).
(ProxyActor pid=60476) INFO 2025-08-02 16:33:10,026 proxy 127.0.0.1 -- Got updated endpoints: {Deployment(name='MySentimentModel', app='default'): EndpointInfo(route='/', app_is_cross_language=False)}.
(ProxyActor pid=60476) INFO 2025-08-02 16:33:10,036 proxy 127.0.0.1 -- Started <ray.serve._private.router.SharedRouterLongPollClient object at 0x

DeploymentHandle(deployment='MySentimentModel')

(ServeReplica:default:MySentimentModel pid=60479) INFO 2025-08-02 16:35:07,451 default_MySentimentModel xfbcpg1v ad44d678-d468-474e-934c-3fc989df3af5 -- GET /predict 200 1131.0ms
(ServeReplica:default:MySentimentModel pid=60479) INFO 2025-08-02 16:37:00,868 default_MySentimentModel xfbcpg1v 4d8caf0e-f709-46fc-b7b3-d0c4e85b1d10 -- GET /predict 200 61.2ms
(ServeReplica:default:MySentimentModel pid=60478) INFO 2025-08-02 16:37:37,338 default_MySentimentModel yb74mu73 a3df9ceb-3ebe-464c-b92a-e6ea260360fb -- GET /predict 200 1238.7ms
(ServeReplica:default:MySentimentModel pid=60478) INFO 2025-08-02 16:38:06,523 default_MySentimentModel yb74mu73 ad5e9c52-96d3-4e39-83ac-9a2378cbb711 -- GET /predict 200 568.3ms


In [6]:
def get_response(text_payload :str):
    response = requests.get("http://localhost:8000/predict", params={"text": text_payload})
    return response.json()

In [7]:
get_response("There are libraries built on top of Ray")

{'text': 'there are libraries built on top of ray',
 'sentiment': [{'label': 'POSITIVE', 'score': 0.8838603496551514}]}

In [8]:
get_response("Edinburgh has a buzzing ML community")

{'text': 'edinburgh has a buzzing ml community',
 'sentiment': [{'label': 'POSITIVE', 'score': 0.9889107942581177}]}

In [9]:
get_response("Too much heat drains me")

{'text': 'too much heat drains me',
 'sentiment': [{'label': 'NEGATIVE', 'score': 0.999446451663971}]}